In [2]:
{-# LANGUAGE OverloadedStrings #-}

import AdventOfCode
import qualified Data.Vector as V
import qualified Data.Attoparsec.ByteString.Char8 as A

input <- dayLines 8

In [4]:
import Data.Functor (($>))

data Instruction = Jmp | Acc | Nop deriving (Eq, Show)

parseInstr = do
    instr <- A.choice
        [ "jmp" $> Jmp
        , "acc" $> Acc
        , "nop" $> Nop
        ]
    A.skipSpace
    arg <- A.signed A.decimal
    pure (instr, arg)

instructions = V.fromList $ map (parsed parseInstr) input

In [6]:
data MachineState = MachineState
    { msPointer :: Int
    , msAccumulator :: Int
    , msInstructions :: V.Vector (Instruction, Int)
    } deriving (Eq, Show)

stepInstruction :: MachineState -> MachineState
stepInstruction ms = let
    pointer = msPointer ms
    currInstr = msInstructions ms V.! pointer
    ms' = case currInstr of
        (Nop, _) -> ms { msPointer = pointer + 1 }
        (Acc, v) -> ms
            { msPointer = pointer + 1
            , msAccumulator = msAccumulator ms + v
            }
        (Jmp, o) -> ms { msPointer = pointer + o}
    in ms'

In [13]:
import qualified Data.Set as S
import Debug.Trace

findAccumulatorBeforeRepeat :: S.Set Int -> MachineState -> Int
findAccumulatorBeforeRepeat pointers ms = let
    currPointer = msPointer ms
    in if currPointer `S.member` pointers
        then msAccumulator ms
        else let
            pointers' = S.insert currPointer pointers
            ms' = stepInstruction ms
            in findAccumulatorBeforeRepeat pointers' ms'

In [14]:
findAccumulatorBeforeRepeat S.empty $ MachineState 0 0 instructions

2058